# **Credit prediction (AI 13 안도윤)**
신용 카드 신청자가 제출한 개인 정보 및 데이터를 사용하여 
***향후 채무 불이행 및 신용 카드 대출 가능성을 예측***

---




> ## **1.** 데이터 세부사항


* ### **링크**  
  https://www.kaggle.com/datasets/rikdifos/credit-card-approval-prediction
  

* ### **고객정보 csv 파일** (application_record.csv)
ID:	클라이언트 번호	
CODE_GENDER:	성별	
FLAG_OWN_CAR:	차 소유 여부 \
FLAG_OWN_REALTY:	부동산 소유 여부 \
CNT_CHILDREN:	자녀의 수	
AMT_INCOME_TOTAL:	연간 소득	
NAME_INCOME_TYPE:	소득 카테고리	
NAME_EDUCATION_TYPE:	교육 수준	
NAME_FAMILY_STATUS:	결혼 여부	
NAME_HOUSING_TYPE:	가정 타입 (아파트, 렌트, 부모동거 등)	
DAYS_BIRTH:	생일	(현재 날짜(0)에서 거꾸로 계산, -1은 어제)\
DAYS_EMPLOYED:	고용 시작일	(현재 날짜(0)부터 거꾸로 계산, 양성이면 현재 실직 중인 사람)\
FLAG_MOBIL:	휴대 전화 유무\
FLAG_WORK_PHONE:	직장 전화 유무\
FLAG_PHONE:	전화기 유무\
FLAG_EMAIL:	이메일 유무\
OCCUPATION_TYPE:	직업	
CNT_FAM_MEMBERS:	가족 구성원 수

* ### **연체여부 csv 파일**(Credit_record.csv)
ID:	클라이언트 번호	
MONTHS_BALANCE:	기록월	(추출된 데이터의 월은 시작점, 역순, 0은 현재 월, -1은 이전 월)
STATUS:	상태	(0: 1-29일 연체 1: 30-59일 연체 2: 60-89일 연체 3: 90-119일 연체 4: 연체 120-149일 5: 연체 또는 불량 부채, 상각 150일 C: 해당 월 상환 X: 해당 월 대출 없음)











> ## **2.** 데이터 전처리
### 2-1. 데이터셋 로드



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd

df1 = pd.read_csv('/content/drive/MyDrive/codestates/archive (2)/application_record.csv')
df2 = pd.read_csv('/content/drive/MyDrive/codestates/archive (2)/credit_record.csv')

In [ ]:
df_ap = df1.copy()
df_cr = df2.copy()

### 2-2. 데이터 정보

In [ ]:
!pip install category_encoders

In [ ]:
#크기
df_ap.shape, df_cr.shape

In [ ]:
#누락값 개수
df_ap.isnull().sum()

In [ ]:
df_cr.isnull().sum()

> ## **3.** 데이터 Wrangling
### 3-1. 데이터 cleaning 및 변환


*   df_ap (고객 정보 데이터)





In [ ]:
df_ap.head(10)
#아이디만 다른 중복 행 여러개 존재

In [ ]:
#중복값 확인
print(df_ap['ID'].nunique())
print(df_cr['ID'].nunique())

In [ ]:
#두 데이터셋의 교집합 갯수
len(set(df_ap['ID']).intersection(set(df_cr['ID'])))

In [ ]:
#고객 정보 데이터셋의 중복 데이터 제거 
df_ap = df_ap.drop_duplicates('ID', keep='last') 

#고객 정보 데이터셋 Occupation type 제거 (null 값 많기 때문)
df_ap.drop('OCCUPATION_TYPE', axis=1, inplace=True) 



*  df_cr (고객별 대출 상환 내역)



In [ ]:
df_cr.head(10)

In [ ]:
#months balance 양수값으로 변환
df_cr['Months_current'] = df_cr['MONTHS_BALANCE'] * (-1) 
df_cr.drop('MONTHS_BALANCE', axis=1, inplace=True)



---


은행의 입장에서 고객의 카드 연체율은 리스크이기 때문에 고객의 연체 여부를 알기 위한 머신러닝 모델 생성
-> Target 값 : 고객의 STATUS

In [ ]:
#STATUS 정수로 변환
def to_int(x):
    if x == '0':
        return 1 # 1~29일 연체 
    elif x == '1':
        return 2 # 30~59일 연체  
    elif x == '2':
        return 3 # 60~89일 연체 
    elif x == '3':
        return 4 # 90~119일 연체  
    elif x == '4':
        return 5 # 120~149일 연체 
    elif x == '5':
        return 6 # 150일 이상 연체 
    else:
        return 0 # 상환완료 혹은 대출 경험 없는 경우

In [ ]:
df_cr['status'] = df_cr['STATUS'].apply(to_int)

In [ ]:
df_cr.head(5)



---


[신용 등급 column 생성]



*   연체 경험 없음 : 0 
*   한 달 이상 연체 고객 : 1 \








In [ ]:
# 한달 이상 연체가 된 경험이 있는 경우 1 / 그런 적이 한번도 없는 경우 0
def credit(y):
    if (2 or 3 or 4 or 5 or 6) in y :
        return 1
    else:
        return 0

#새로운 dataframe 형성
df_cr2 = pd.DataFrame(df_cr.groupby('ID')['status'].apply(list))

In [ ]:
df_cr2['Credit'] = pd.DataFrame(df_cr.groupby('ID')['status'].apply(list))['status'].apply(credit)
df_cr2.head()

In [ ]:
df_cr2 = df_cr2.reset_index().drop('status', axis=1)

In [ ]:
df_cr2.value_counts(subset='Credit')

In [ ]:
# credit 분포 확인
import seaborn as sns
print(df_cr2['Credit'].value_counts(normalize = True))
sns.countplot(x=df_cr2['Credit']);

In [ ]:
df_cr2.head()

### 3-2. 데이터프레임 merge

In [ ]:
df_m = df_ap.merge(df_cr2, how='inner', on='ID')

In [ ]:
df_m.head()

> ## **4.** EDA & Feature Engineering


In [ ]:
df = df_m.copy()
df.head()

In [ ]:
df.info()

In [ ]:
df.describe().T

In [ ]:
#mobile phone 여부는 모두 1이기 때문에 column 삭제
df = df.drop(['FLAG_MOBIL'], axis=1)
df.head(5)

In [ ]:
# 컬럼명 수정
df.rename(columns={'CODE_GENDER':'GENDER',
                   'FLAG_OWN_CAR':'CAR',
                   'FLAG_OWN_REALTY':'REALTY',
                   'CNT_CHILDREN':'CHILDREN',
                   'AMT_INCOME_TOTAL':'ANNUAL_INCOME',
                   'NAME_INCOME_TYPE':'INCOME_TYPE',
                   'NAME_EDUCATION_TYPE':'EDUCATION_LEVEL',
                   'NAME_HOUSING_TYPE':'HOUSING_TYPE',
                   'NAME_FAMILY_STATUS':'MARRIED',
                   'FLAG_WORK_PHONE':'PHONE_WORK',
                   'FLAG_PHONE':'PHONE_HOME',
                   'FLAG_EMAIL':'EMAIL',
                   'CNT_FAM_MEMBERS':'FAMILY_SIZE',
                   'Credit':'CREDIT'
                   }, inplace=True)
df.head(3)

In [ ]:
# 이진형 column은 숫자로 변환

#남자 1 여자 0
df['GENDER'] = df['GENDER'].apply(lambda x : 0 if x == 'M' else 1)
#차 있음 1 없음 0
df['CAR'] = df['CAR'].apply(lambda x : 0 if x == 'N' else 1)
#부동산 소유 1 없음 0
df['REALTY'] = df['REALTY'].apply(lambda x : 0 if x == 'N' else 1)

In [ ]:
# DAYS_BIRTH 나이로 전환
df['AGE'] = -df['DAYS_BIRTH'] // 365

# DAYS_EMPLOYED' 양수로 전환
df['DAYS_EMPLOYED'] = df['DAYS_EMPLOYED'] * (-1)

df.drop('DAYS_BIRTH',axis = 1 , inplace = True)

In [ ]:
df.info()



---


## 4-1 데이터 이상치 파악 및 제거


*  수치형 column







In [ ]:
#데이터 형식별 이상치 확인 및 제거

nums = ['CHILDREN', 'ANNUAL_INCOME', 'DAYS_EMPLOYED', 'PHONE_WORK', 'PHONE_HOME', 'EMAIL',
       'FAMILY_SIZE','AGE']
       
cats= ['INCOME_TYPE', 'EDUCATION_LEVEL', 'MARRIED', 'HOUSING_TYPE']


#Outlier 확인 (Numeric Feature)
import matplotlib.pyplot as plt
import seaborn as sns

for i in nums:
    df[i].plot()
    plt.title(i)
    plt.show()
    print('0보다 큰 샘플 :', len(df[df[i] > 0]))
    print('0과 같은 샘플 :', len(df[df[i] == 0]))
    print('0보다 작은 샘플 :', len(df[df[i] < 0]))



> DAYS_EMPLOYED column 에 음수값 존재. 이상치 있는 column 확인 가능



In [ ]:
# DAYS_EMPLOYED 음수값은 일을 하고 있지 않은 상태로, 0으로 통일

df.loc[df['DAYS_EMPLOYED'] < 0, 'DAYS_EMPLOYED'] = 0



> 이상치가 있을 가능성이 높은 column : 


*  CHILDREN
*  ANNUAL_INCOME
*  AGE
*  DAYS_EMPLOYED
*  FAMILY_SIZE

-> Boxplot으로 정확하게 시각화 진행





In [ ]:
# boxplot으로 Outlier 보기
fig = plt.figure(figsize=(15,15)) 
for i, col in enumerate(['CHILDREN','ANNUAL_INCOME','AGE','DAYS_EMPLOYED','FAMILY_SIZE']):
  plt.subplot(3,2,i+1)
  sns.boxplot(df[col])

In [ ]:
#Outlier 제거 (numeric column)
#범위 (min: 25%, max: 75%)
def remove_outlier(df):
  for i in ['CHILDREN','ANNUAL_INCOME','AGE','DAYS_EMPLOYED','FAMILY_SIZE']:
    df = df[(df[i]>=(df[i].quantile(0.25))) & (df[i]<=(df[i].quantile(0.75)))]
  return df

remove_outlier(df)

In [ ]:
df.reset_index(drop=True)




---


*   명목형 column outlier 확인




In [ ]:
#categorical feature 이상치 확인
for i in cats :
   print(i,'\n')
   print(df[i].value_counts())

In [ ]:
df.drop('ID', axis=1, inplace=True) # ID 제거

In [ ]:
df_model = df.copy()

In [ ]:
df_model.shape

In [ ]:
#credit(target) 분포 확인
df_model['CREDIT'].value_counts(normalize=True)

In [ ]:
sns.countplot(data=df_model, x='CREDIT');

## 4-2 EDA, 시각화
### 4-2-1 자료 시각화 (이진형 데이터)




In [ ]:
df_sort = df.copy()
df_sort = df_sort.sort_values('ANNUAL_INCOME')
df_sort = df_sort.reset_index(drop=True)
df_sort.ID=df_sort.index
df_sort

In [ ]:
#전체 고객목록 중 2달 이상 연체 경험이 있는 고객 비율
print('36457명의 고객 중\n'+str(round(df_sort.CREDIT.sum()/len(df_sort)*100,2)) +'% ,',str(df_sort.CREDIT.sum()) +' 명의 2달 이상 연체 고객이 있음' )

In [ ]:
#이진형 데이터 (성별, 자동차 유무, 부동산 유뮤, 전화소유 유무, 이메일 유무) 에 따른 신용도 분류
binary_features = ['GENDER', 'CAR', 'REALTY', 'PHONE_WORK', 'PHONE_HOME', 'EMAIL']
binary_df = df_sort[binary_features+['CREDIT']]
dict_list = []
for feature in binary_features:
    for one_type in [0, 1]:
        dict_list.append({'feature': feature,
                          'type': one_type,
                          'credit_rate': len(binary_df[binary_df[feature]==one_type][binary_df.CREDIT==1])/len(binary_df[binary_df[feature]==one_type]),
                          'count': len(binary_df[binary_df[feature]==one_type]),
                          'Credit_count': len(binary_df[binary_df[feature]==one_type][binary_df.CREDIT==1])
                         })

group_binary = pd.DataFrame.from_dict(dict_list)
sns.barplot(x="feature", y="credit_rate", hue="type", data=group_binary)
plt.show()
group_binary



> **성별, 차량 소유, 부동산 소유, 전화 소유, 이메일 소유 등은 신용 등급에 큰 영향을 미치지 않다는 것을 알 수 있음**



*  수입이 많은 사람이 대출 상환 능력이 높다고 생각함.\
전체 수입에 따른 credit 등급 분류 진행

In [ ]:
sns.boxplot(x="CREDIT", y="ANNUAL_INCOME", data=df_sort[df_sort.ANNUAL_INCOME <=500000])
plt.show()



> 연간 수입도 데이터 상으로는 신용도에 큰 영향을 미치지 않는다는 것을 알 수 있음





---
### 4-2-2 자료 시각화 (명목형 데이터)




*   거주하는 집의 형태에 따른 신용도



In [ ]:
total=df_sort.groupby(by=['HOUSING_TYPE']).size().reset_index(name='total')
credit = df[df_sort.CREDIT==1].groupby(by=['HOUSING_TYPE']).size().reset_index(name='credit_count')
credit_rate = pd.merge(total, credit, how='outer', on=['HOUSING_TYPE']).fillna(0)
credit_rate['credit_rate'] = credit_rate.credit_count / credit_rate.total
sns.barplot(x="HOUSING_TYPE", y="credit_rate", data=credit_rate)
plt.xticks(rotation=90)
plt.show()
credit_rate


    거주 형태도 신용도에 큰 영향을 미치지 않음




*   교육 형태 (최종 학력)에 따른 신용도




In [ ]:
total=df_sort.groupby(by=['EDUCATION_LEVEL']).size().reset_index(name='total')
credit = df[df_sort.CREDIT==1].groupby(by=['EDUCATION_LEVEL']).size().reset_index(name='credit_count')
credit_rate = pd.merge(total, credit, how='outer', on=['EDUCATION_LEVEL']).fillna(0)
credit_rate['credit_rate'] = credit_rate.credit_count / credit_rate.total
sns.barplot(x="EDUCATION_LEVEL", y="credit_rate", data=credit_rate)
plt.xticks(rotation=90)
plt.show()
credit_rate

      최종 학력은 초등 교육인 사람들의 저신용도 비율이 제일 많음



*   수입 형태에 따른 신용도


In [ ]:
total=df_sort.groupby(by=['INCOME_TYPE']).size().reset_index(name='total')
credit = df[df_sort.CREDIT==1].groupby(by=['INCOME_TYPE']).size().reset_index(name='credit_count')
credit_rate = pd.merge(total, credit, how='outer', on=['INCOME_TYPE']).fillna(0)
credit_rate['credit_rate'] = credit_rate.credit_count / credit_rate.total
sns.barplot(x="INCOME_TYPE", y="credit_rate", data=credit_rate)
plt.xticks(rotation=90)
plt.show()
credit_rate

     직업이 따로 없고 학생인 사람들의 저신용도 비율이 높음



*   결혼 여부에 따른 신용도




In [ ]:
total=df_sort.groupby(by=['MARRIED']).size().reset_index(name='total')
credit = df[df_sort.CREDIT==1].groupby(by=['MARRIED']).size().reset_index(name='credit_count')
credit_rate = pd.merge(total, credit, how='outer', on=['MARRIED']).fillna(0)
credit_rate['credit_rate'] = credit_rate.credit_count / credit_rate.total
sns.barplot(x="MARRIED", y="credit_rate", data=credit_rate)
plt.xticks(rotation=90)
plt.show()
credit_rate

     배우자를 떠나보낸 사람들의 저신용도 비율이 높으나, 차이는 다른 항목과 미미함

> ## **5.** Modeling


### 5-1. Train, Test 데이터 나누기




In [ ]:
#train test val 나누기
target = 'CREDIT'
features = df_model.drop(columns=[target], axis=1).columns

In [ ]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(df_model, test_size=0.2, stratify=df[target], random_state=42)
train, val = train_test_split(train, test_size=0.2, stratify=train[target], random_state=42)

In [ ]:
train.shape, val.shape, test.shape

In [ ]:
X_train = train[features]
y_train = train[target]

X_val = val[features]
y_val = val[target]

X_test = test[features]
y_test = test[target]

X_train.shape, y_train.shape, X_val.shape, y_val.shape, X_test.shape, y_test.shape



---


## 5-2. 하이퍼파라미터 설정 없이 모델 학습


5-2-1. Baseline 설정



In [ ]:
y_train.value_counts(normalize=True)

In [ ]:
major = 1
y_pred_base = [major] * len(y_val)

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, classification_report

print(f'accuracy_score : {accuracy_score(y_val, y_pred_base)}')
print(f'f1_score : {f1_score(y_val, y_pred_base)}')
print(f'auc_score : {roc_auc_score(y_val, y_pred_base)}')
print(classification_report(y_val, y_pred_base))




> 데이터 불균형이 발생하여 f1 score 값이 매우 낮게 나옴
####  **데이터 불균형 문제를 해결해야 됨.**








> 데이터 불균형 문제 해결 방법


1.   Under Sampling : 데이터 손상의 가능성이 크다.
2.   Over Sampling\
     --> ADASYN는 속도 저하의 단점이 있음.\
     --> 실무에서 많이 사용하는 SMOTE 이용할 것입니다.






---


### 5-2-2 데이터 불균형 해결 (SMOTE:Oversampling)




In [ ]:
X_train.info()

In [ ]:
!pip install -U imbalanced-learn

In [ ]:
from imblearn.over_sampling import SMOTENC # Data Imbalance 해결

smoter = SMOTENC(categorical_features=[0,1,2,3,5,6,7,8,9,10,11,12,14], random_state=42, n_jobs=-1)

X_train_over, y_train_over = smoter.fit_resample(X_train, y_train)
X_val_over, y_val_over = smoter.fit_resample(X_val, y_val)
X_test_over, y_test_over = smoter.fit_resample(X_test, y_test)

X_train = pd.DataFrame(X_train_over, columns=X_train.columns)
X_val = pd.DataFrame(X_val_over, columns=X_train.columns)
X_test = pd.DataFrame(X_test_over, columns=X_train.columns)

y_train = y_train_over
y_val = y_val_over
y_test = y_test_over


In [ ]:
X_train = pd.DataFrame(X_train_over, columns=X_train.columns)
X_val = pd.DataFrame(X_val_over, columns=X_train.columns)
X_test = pd.DataFrame(X_test_over, columns=X_train.columns)

y_train = y_train_over
y_val = y_val_over
y_test = y_test_over

print('SMOTE 적용 후 학습용 피처/레이블 데이터:', X_train.shape, y_train.shape)
print('SMOTE 적용 후 val 피처/레이블 데이터:', X_val.shape, y_val.shape)
print('SMOTE 적용 후 레이블 값 분포: \n', pd.Series(y_train).value_counts())




---


*  Oversampling 후 Baseline 확인하기



In [ ]:
y_train.value_counts(normalize=True)

In [ ]:
major1 = 1
y_pred_base1 = [major1] * len(y_val)

In [ ]:
print(f'Oversampling 후 accuracy_score : {accuracy_score(y_val, y_pred_base1)}')
print(f'Oversampling 후 f1_score : {f1_score(y_val, y_pred_base1)}')
print(f'Oversampling 후 auc_score : {roc_auc_score(y_val, y_pred_base1)}')
print(classification_report(y_val, y_pred_base1))

# 데이터 불균형을 위한 SMOTE 사용으로 accuracy score 가 0.5로 나옴



---


### 5-2-3 DecisionTreeClassification

In [ ]:
from sklearn.pipeline import make_pipeline
from category_encoders import OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.tree import DecisionTreeClassifier

model_DT = make_pipeline(OrdinalEncoder(),
                         SimpleImputer(),
                         DecisionTreeClassifier())

model_DT.fit(X_train, y_train)

In [ ]:
#train
from sklearn.metrics import plot_confusion_matrix
y_pred_train = model_DT.predict(X_train)

print(f'accuracy_score : {accuracy_score(y_train, y_pred_train)}')
print(f'f1_score : {f1_score(y_train, y_pred_train)}')
print(f'auc_score : {roc_auc_score(y_train, y_pred_train)}')

print(classification_report(y_train, y_pred_train))

In [ ]:
#validation
y_pred_val = model_DT.predict(X_val)

print(f'accuracy_score : {accuracy_score(y_val, y_pred_val)}')
print(f'f1_score : {f1_score(y_val, y_pred_val)}')
print(f'auc_score : {roc_auc_score(y_val, y_pred_val)}')
print(classification_report(y_val, y_pred_val))



---
### 5-2-4 RandomForestClassification


In [ ]:
from sklearn.ensemble import RandomForestClassifier

model_RF = make_pipeline(OrdinalEncoder(),
                         SimpleImputer(),
                         RandomForestClassifier())

model_RF.fit(X_train, y_train)

In [ ]:
#train
y_pred_train = model_RF.predict(X_train)

print(f'accuracy_score : {accuracy_score(y_train, y_pred_train)}')
print(f'f1_score : {f1_score(y_train, y_pred_train)}')
print(f'auc_score : {roc_auc_score(y_train, y_pred_train)}')
print(classification_report(y_train, y_pred_train))

In [ ]:
#validation
y_pred_val = model_RF.predict(X_val)

print(f'accuracy_score : {accuracy_score(y_val, y_pred_val)}')
print(f'f1_score : {f1_score(y_val, y_pred_val)}')
print(f'auc_score : {roc_auc_score(y_val, y_pred_val)}')
print(classification_report(y_val, y_pred_val))



---


### 5-2-5 GradientBoostingClassification

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

model_GB = make_pipeline(OrdinalEncoder(),
                        SimpleImputer(),
                        GradientBoostingClassifier())

model_GB.fit(X_train, y_train)

In [ ]:
#train
y_pred_train = model_GB.predict(X_train)

print(f'accuracy_score : {accuracy_score(y_train, y_pred_train)}')
print(f'f1_score : {f1_score(y_train, y_pred_train)}')
print(f'auc_score : {roc_auc_score(y_train, y_pred_train)}')
print(classification_report(y_train, y_pred_train))

In [ ]:
#validation
y_pred_val = model_GB.predict(X_val)

print(f'accuracy_score : {accuracy_score(y_val, y_pred_val)}')
print(f'f1_score : {f1_score(y_val, y_pred_val)}')
print(f'auc_score : {roc_auc_score(y_val, y_pred_val)}')
print(classification_report(y_val, y_pred_val))


---


### 5-2-6 XGBClassification

In [ ]:
import xgboost
from xgboost import XGBClassifier

model_XGB = make_pipeline(OrdinalEncoder(),
                          SimpleImputer(),
                          XGBClassifier())

model_XGB.fit(X_train, y_train)

In [ ]:
#train
y_pred_train = model_XGB.predict(X_train)

print(f'accuracy_score : {accuracy_score(y_train, y_pred_train)}')
print(f'f1_score : {f1_score(y_train, y_pred_train)}')
print(f'auc_score : {roc_auc_score(y_train, y_pred_train)}')
print(classification_report(y_train, y_pred_train))

In [ ]:
#validation
y_pred_val = model_XGB.predict(X_val)

print(f'accuracy_score : {accuracy_score(y_val, y_pred_val)}')
print(f'f1_score : {f1_score(y_val, y_pred_val)}')
print(f'auc_score : {roc_auc_score(y_val, y_pred_val)}')
print(classification_report(y_val, y_pred_val))

---


## 5-3. 하이퍼파라미터 조정 후 모델별 성능 비교

- 하이퍼파라미터 조정 없이 모델링 진행했을 때, GradientBoosting과 XGB 가 성능이 좋았음.
- XGB로 하이퍼파라미터 조정 후 모델링 진행

###5-3-1 XGBClassification

In [ ]:
model_XGB = make_pipeline(OrdinalEncoder(),
                          SimpleImputer(),
                          XGBClassifier(random_state=42))

model_XGB.fit(X_train, y_train)

In [ ]:
y_pred_train = model_XGB.predict(X_train)

print(f'accuracy_score : {accuracy_score(y_train, y_pred_train)}')
print(f'f1_score : {f1_score(y_train, y_pred_train)}')
print(f'auc_score : {roc_auc_score(y_train, y_pred_train)}')
print(classification_report(y_train, y_pred_train))

In [ ]:
y_pred_val = model_XGB.predict(X_val)

print(f'accuracy_score : {accuracy_score(y_val, y_pred_val)}')
print(f'f1_score : {f1_score(y_val, y_pred_val)}')
print(f'auc_score : {roc_auc_score(y_val, y_pred_val)}')
print(classification_report(y_val, y_pred_val))



> RandomizedSearchCV 를 이용한 최적의 하이퍼파라미터 찾기



In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

params_dist = {
    'xgbclassifier__max_depth':randint(3, 10),
    'xgbclassifier__learning_rate':[0.1,0.2,0.3],
    'xgbclassifier__n_estimators':randint(100,1000,50)
    }

search_Rd = RandomizedSearchCV(model_XGB,
                            param_distributions=params_dist,
                            n_iter=50,
                            cv=3,
                            n_jobs=-1,
                            verbose=1,
                            scoring='accuracy' # accuracy 기준
                            )

search_Rd.fit(X_train, y_train)

In [ ]:
print('최고 예측 정확도: {0:.4f}'.format(search_Rd.best_score_))
print('최적 하이퍼 파라미터:\n',search_Rd.best_params_)



> 최적의 하이퍼파라미터로 학습



In [ ]:
from sklearn.pipeline import Pipeline
model_final = Pipeline([
                        ('preprocessing', make_pipeline(OrdinalEncoder(), SimpleImputer())),
                          ('xgb', XGBClassifier(learning_rate=0.2, xgbclassifier__max_depth=7, xgbclassifier__n_estimators=802, random_state=42))
])
model_final.fit(X_train, y_train)

y_pred= model_final.predict(X_test)


In [ ]:
y_pred_val = model_final.predict(X_val)

print(f'accuracy_score : {accuracy_score(y_val, y_pred_val)}')
print(f'f1_score : {f1_score(y_val, y_pred_val)}')
print(f'auc_score : {roc_auc_score(y_val, y_pred_val)}')
print(classification_report(y_val, y_pred_val))

In [ ]:
print(f'accuracy_score : {accuracy_score(y_test, y_pred)}')
print(f'f1_score : {f1_score(y_test, y_pred)}')
print(f'auc_score : {roc_auc_score(y_test, y_pred)}')
print(classification_report(y_test, y_pred))

In [ ]:
#ROC curve
from sklearn.metrics import roc_curve

y_pred_val_proba = model_final.predict_proba(X_val)[:,1]

fpr, tpr, thresholds = roc_curve(y_val, y_pred_val_proba)

roc = pd.DataFrame({
    'FPR(Fall-out)': fpr, 
    'TPRate(Recall)': tpr, 
    'Threshold': thresholds
})
roc

In [ ]:
plt.scatter(fpr, tpr)
plt.title('ROC curve')
plt.xlabel('FPR(Fall-out)')
plt.ylabel('TPR(Recall)');

In [ ]:
#threshold
import numpy as np
optimal_idx = np.argmax(tpr - fpr)
optimal_threshold = thresholds[optimal_idx]

print('idx:', optimal_idx, ', threshold:', optimal_threshold)

In [ ]:
y_pred_optimal = y_pred_val_proba >= optimal_threshold
print(classification_report(y_val, y_pred_optimal))

In [ ]:
y_pred_test_proba = model_final.predict_proba(X_test)[:,1]

y_pred_optimal = y_pred_test_proba >= optimal_threshold
print(classification_report(y_test, y_pred_optimal))

> ## **6.** Permutation Importance

In [ ]:
!pip install eli5
import eli5
from eli5.sklearn import PermutationImportance

In [ ]:
permuter = PermutationImportance(
    model_final.named_steps['xgb'],
    scoring='accuracy',
    n_iter=5,
    random_state=42
)

X_test_t = model_final.named_steps['preprocessing'].transform(X_test)

permuter.fit(X_test_t, y_test)

In [ ]:
feature_names = X_test.columns.tolist()
pd.Series(permuter.feature_importances_, feature_names).sort_values(ascending=False)

In [ ]:
eli5.show_weights(
    permuter, 
    top=None, 
    feature_names=feature_names 
)

> ## **7.** PDP(Partial Dependence Plot)

In [ ]:
!pip install pdpbox
!pip install shap

In [ ]:
from pdpbox import pdp
from pdpbox.pdp import pdp_isolate, pdp_plot
from pdpbox.pdp import pdp_interact, pdp_interact_plot

import shap

In [ ]:
feature = 'FAMILY_SIZE'

isolated = pdp_isolate(
    model=model_final, 
    dataset=X_val, 
    model_features=X_val.columns, 
    feature=feature
)

pdp_plot(isolated, feature_name=feature)


      <가족 구성원>
      1. 2.5명 이하 : 2개월 이상 연체 가능성 낮음
      2. 약 2.5명 ~ 3명 이하 : 연체 가능성 증가
      3. 3명 ~ 20명 : 연체 가능성 미미하게 증가

In [ ]:
feature = 'ANNUAL_INCOME'

isolated = pdp_isolate(
    model=model_final, 
    dataset=X_val, 
    model_features=X_val.columns, 
    feature=feature
)

pdp_plot(isolated, feature_name=feature)


    <연간 수입>
    수입이 많은 사람들은 연체 가능성 낮음

In [ ]:
feature = 'CHILDREN'

isolated = pdp_isolate(
    model=model_final, 
    dataset=X_val, 
    model_features=X_val.columns, 
    feature=feature
)

pdp_plot(isolated, feature_name=feature)

    <자녀의 수>
    자녀가 많을수록 연체 가능성이 낮아짐

In [ ]:
feature = 'AGE'

isolated = pdp_isolate(
    model=model_final, 
    dataset=X_test, 
    model_features=X_test.columns, 
    feature=feature
)

pdp_plot(isolated, feature_name=feature)

    <나이>
    1. 20대 후반까지 연체 가능성이 있음
    2. 30이 넘어가서부터 연체 가능성 감소
    3. 60대 이후 : 연체 가능성 대폭 감소

In [ ]:
feature = 'DAYS_EMPLOYED'

isolated = pdp_isolate(
    model=model_final, 
    dataset=X_test, 
    model_features=X_test.columns, 
    feature=feature
)

pdp_plot(isolated, feature_name=feature)

    <재직 기간>
    1. 5년 이하 : 연체 가능성 낮은 편
    2. 5년 이상~7년 이하 : 연체 가능성 대폭 감소함
    3. 7년 이상~9년반 이하 : 연체 가능성 대폭 증가함
    4. 10년 이상 : 연체 가능성 점점 감소



---
##7-1 PDP interact (성별과 나이)


In [ ]:
from pdpbox.pdp import pdp_interact, pdp_interact_plot
features = ['GENDER', 'AGE']

interaction = pdp_interact(
    model=model_final, 
    dataset=X_test, 
    model_features=X_test.columns, 
    features=features
)

pdp_interact_plot(interaction, plot_type='grid', feature_names=features);

#0 : 남자, 1 : 여자

     성별에 따른 연체 가능성은 차이가 미미함.
     나이가 들수록 연체 가능성이 감소함.
     21~29세의 연체율이 제일 크다.

> ## **8.** SHAP

In [ ]:
X_test.shape

In [ ]:
X_test_enc = enc.transform(X_test)

shap.initjs()
explainer = shap.TreeExplainer(model_final.named_steps['xgb'])
shap_values = explainer.shap_values(X_test_enc.iloc[:2000])
shap.force_plot(explainer.expected_value, shap_values, features=X_test_enc.iloc[:2000]);


In [ ]:
shap.summary_plot(shap_values, X_test_enc.iloc[:2000], plot_type="violin");

    1. 나이가 어릴수록 연체 가능성이 높아짐
    2. 결혼 상태 feature 값이 작을수록 연체 가능성이 높아짐
    (위에 자료 시각화에서 widow와 single 이 연체 가능성이 높았음)

In [ ]:
shap.summary_plot(shap_values, X_test_enc.iloc[:2000], plot_type="bar");

결론 : 결혼을 하지 않은 싱글일수록, 30대 이하가 연체 가능성이 높으므로
카드 발급을 할 때 나이 항목과 결혼 여부를 확인할 필요가 있음.